# Load processed datasets
From datasets.ipynb

In [1]:
import pickle

path = '/storage2/mamille3/hegemonic_hate/tmp/processed_datasets.pkl'
with open(path, 'rb') as f:
    hate_datasets = pickle.load(f)
hate_datasets.keys()

dict_keys(['cad', 'elsherief2021', 'sbic', 'kennedy2020', 'salminen2018'])